In [55]:
from requests import request
from bs4 import BeautifulSoup

from math import log2
import pandas as pd
import re
from nltk.tokenize import sent_tokenize
from konlpy.tag import Hannanum, Komoran
ma = Komoran()

from collections import defaultdict, Counter

In [ ]:
dom = BeautifulSoup(request('get', 'https://www.goodchoice.kr/product/search/5').text, 'html5lib')
ids = [a.attrs['data-ano'] for a in dom.select('li.list_2 > a[data-ano]')]

In [ ]:
url = 'https://www.goodchioce.kr/product/get_review_non'
reviews = pd.DataFrame(columns=['ano', 'aepreg', 'rating', 'review'])

for i in ids:
    page = 1

    while True:
        resp = request('post', url, data={'page':page, 'ano': ids[0]})
        for item in resp.json()['result']['items']:
            reviews.loc[len(reviews)] = (i, item['aepreg'], item['epilrate'], item['aepcont'])
        if page > resp.json()['total_page']:
            break
        page += 1
# resp.json
# len(resp.json()['result']['items'])

어휘기반 감성분석
-> 어휘(사전적 정의와 다름, 사람이 일일히 정한 것, 코퍼스 - Semi-supervised)
eg. 여기어때 -> 리뷰
1. 리뷰(평점) -> 두 그룹(<5, >5)  -> term ㅌ positive, negetive <- PMI
2.               X   대표단어 셋 -> term ㅌ 대표단어(positive), 대표단어(negetive) <- PMI
3.          -> Topic Model
4.                              -> 개별 토픽들이 어떤 극성(감성)
PMI, PPMI, NPMI + LDA(np, list<-)

In [56]:
with open('reviews.jpg', encoding='utf-8') as fp:
    reviews = pd.read_csv(fp)

reviews[reviews['rating'] < 10].describe()

,Unnamed: 0,ano,aepreg,rating
count,2138.000000,2138.000000,2.138000e+03,2138.000000
mean,3672.119738,56529.226848,1.611976e+09,6.657343
std,2053.632998,11729.379197,3.977837e+07,2.602609
min,0.000000,10681.000000,1.488162e+09,1.000000
25%,2053.750000,47204.000000,1.584905e+09,5.300000
50%,3764.000000,57279.000000,1.617594e+09,7.600000
75%,5502.500000,64461.000000,1.644820e+09,8.600000
max,7035.000000,72146.000000,1.668556e+09,9.600000


In [57]:
def cleanning1(org):
    text = re.sub(r'[^0-9A-Za-zㄱ-ㅎ가-힣.,!? ]+', ' ', org)
    result = list()
    for s in sent_tokenize(text):
        result.append(' '.join([t for t in ma.morphs(s) if len(t) > 1]))
    return ('\n'.join(result)).strip()

In [58]:
reviews['clean1'] = reviews['review'].apply(cleanning1)

In [59]:
negative = reviews[reviews['rating'] < 7]
positive = reviews[reviews['rating'] >= 7]

In [60]:
pTDM = defaultdict(Counter)
nTDM = defaultdict(Counter)

In [61]:
for i, review in enumerate(negative['clean1']):
    for t in review.split():
        nTDM[t][i] += 1

for i, review in enumerate(positive['clean1']):
    for t in review.split():
        pTDM[t][i] += 1

In [62]:
V = list(set(list(nTDM.keys()) + list(pTDM.keys())))

In [63]:
sorted({t: sum(tinfo.values()) for t, tinfo in nTDM.items()}.items(), key=lambda v:v[1], reverse=True)[:5]

[('어요', 982), ('는데', 957), ('습니다', 890), ('어서', 763), ('아서', 715)]

In [64]:
pProb = dict()
length = len(reviews)
for t in V:
    if len(pTDM[t]) > 0:
        pProb[t] = (len((pTDM[t]))+1)/(length+len(V))
    # print(t, len(pTDM[t]))

nProb = dict()
for t in V:
    if len(nTDM[t]) > 0:
        nProb[t] = (len((nTDM[t]))+1)/(length+len(V))

In [66]:
pJoint = dict()
for t in V:
    pJoint[t] = ((len(
        set(list(pTDM[t].keys())).intersection(
        set(list(nTDM[t].keys()))))+1) / (length+len(V)))

P(X,Y) = pJoint
P(X) = pProb
P(Y) = nProb
log2 P(X,Y) / (P(X)*P(Y))

In [68]:
pWords = ['멋있고', '행복한']
nWords = ['화장실', '시설']

pWordsProb = [(len(pTDM[t])+len(nTDM[t]))/length for t in pWords]
nWordsProb = [(len(pTDM[t])+len(nTDM[t]))/length for t in nWords]

In [69]:
pPMI = defaultdict(float)
for t in V:
    pList = set(pTDM[t].keys())
    nList = set(nTDM[t].keys())
    for seed in pWords:
        sList = set(list(pList[t].keys()) + list(nTDM[t].keys()))
        cofreq = len(pList.intersection(nList).intersection(sList))
        jp = (cofreq+1)/(length+len(V))
        pp = (len(pList)+1)/(length+len(V))
        np = (len(nList)+1)/(length+len(V))
        pPMI[t] += log2(jp/(pp*np))

nPMI = defaultdict(float)
for t in V:
    pList = set(pTDM[t].keys())
    nList = set(nTDM[t].keys())
    for seed in nWords:
        sList = set(list(pList[t].keys()) + list(nTDM[t].keys()))
        cofreq = len(pList.intersection(nList).intersection(sList))
        jp = (cofreq+1)/(length+len(V))
        pp = (len(pList)+1)/(length+len(V))
        np = (len(nList)+1)/(length+len(V))
        nPMI[t] += log2(jp/(pp*np))

TypeError: 'set' object is not subscriptable